In [418]:
def document_vector(model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in model.wv.vocab]
    return np.mean(model[doc], axis=0)

In [ ]:
word2vec_.wv.most_similar('extended')

In [ ]:
import pandas as pd
from glob2 import glob

In [ ]:
jsonFiles = glob('../pheme-rnr-dataset/**/source-tweet/*.json') #Can be used absolute or relative paths
print(jsonFiles)

In [ ]:
dfList = []
for jsonFile in jsonFiles:
    df = pd.read_json(jsonFile)
    dfList.append(df)
    
dfTrainingDF = pd.concat(dfList, axis=0)

In [ ]:
dfList = []

for jsonFile in jsonFiles[0:10]:
    df=pd.read_json(jsonFile, orient='index')
    dfList.append(df)

In [ ]:
pd.concat(dfList, axis=1)

In [ ]:
dfTrainingDF

In [ ]:
dfTrainingDF.to_csv()

---

In [ ]:
import json

In [ ]:
data_list=[]
for jsonFile in jsonFiles[0:2]:
    with open(jsonFile) as json_file:
            data_list = json.load(json_file)
    json_file
# data_list
# tweet_data_frame = pd.DataFrame.from_dict(data_list)

In [ ]:
len(data_list)

In [ ]:
jsonFiles

In [ ]:
tweets_data = []
for file in jsonFiles[0:5]:
    
    tweets_file = open(file, "r", encoding = 'utf-8')

    # Read in tweets and store in list: tweets_data
    for line in tweets_file:
        tweet = json.loads(line)
        tweets_data.append(tweet)

    tweets_file.close()

In [ ]:
def flatten_tweets(tweets):
    """ Flattens out tweet dictionaries so relevant JSON is in a top-level dictionary. """
    
    tweets_list = []
    
    # Iterate through each tweet
    for tweet_obj in tweets:
    
        ''' User info'''
        # Store the user screen name in 'user-screen_name'
        tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']
        
        # Store the user location
        tweet_obj['user-location'] = tweet_obj['user']['location']
    
        ''' Text info'''
        # Check if this is a 140+ character tweet
        if 'extended_tweet' in tweet_obj:
            # Store the extended tweet text in 'extended_tweet-full_text'
            tweet_obj['extended_tweet-full_text'] = \
                                    tweet_obj['extended_tweet']['full_text']
    
        if 'retweeted_status' in tweet_obj:
            # Store the retweet user screen name in 
            # 'retweeted_status-user-screen_name'
            tweet_obj['retweeted_status-user-screen_name'] = \
                        tweet_obj['retweeted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['retweeted_status-text'] = \
                                        tweet_obj['retweeted_status']['text']
    
            if 'extended_tweet' in tweet_obj['retweeted_status']:
                # Store the extended retweet text in 
                #'retweeted_status-extended_tweet-full_text'
                tweet_obj['retweeted_status-extended_tweet-full_text'] = \
                tweet_obj['retweeted_status']['extended_tweet']['full_text']
                
        if 'quoted_status' in tweet_obj:
            # Store the retweet user screen name in 
            #'retweeted_status-user-screen_name'
            tweet_obj['quoted_status-user-screen_name'] = \
                            tweet_obj['quoted_status']['user']['screen_name']

            # Store the retweet text in 'retweeted_status-text'
            tweet_obj['quoted_status-text'] = \
                                            tweet_obj['quoted_status']['text']
    
            if 'extended_tweet' in tweet_obj['quoted_status']:
                # Store the extended retweet text in 
                #'retweeted_status-extended_tweet-full_text'
                tweet_obj['quoted_status-extended_tweet-full_text'] = \
                    tweet_obj['quoted_status']['extended_tweet']['full_text']
        
        ''' Place info'''
        if 'place' in tweet_obj:
            # Store the country code in 'place-country_code'
            try:
                tweet_obj['place-country'] = \
                                            tweet_obj['place']['country']
                
                tweet_obj['place-country_code'] = \
                                            tweet_obj['place']['country_code']
                
                tweet_obj['location-coordinates'] = \
                            tweet_obj['place']['bounding_box']['coordinates']
            except: pass
        
        tweets_list.append(tweet_obj)
        
    return tweets_list

In [ ]:
def select_text(tweets):
    ''' Assigns the main text to only one column depending
        on whether the tweet is a RT/quote or not'''
    
    tweets_list = []
    
    # Iterate through each tweet
    for tweet_obj in tweets:
        
        if 'retweeted_status-extended_tweet-full_text' in tweet_obj:
            tweet_obj['text'] = \
                        tweet_obj['retweeted_status-extended_tweet-full_text']
        
        elif 'retweeted_status-text' in tweet_obj:
            tweet_obj['text'] = tweet_obj['retweeted_status-text']
            
        elif 'extended_tweet-full_text' in tweet_obj:
                    tweet_obj['text'] = tweet_obj['extended_tweet-full_text']
                
        tweets_list.append(tweet_obj)
        
    return tweets_list

In [ ]:
# flatten tweets
tweets = flatten_tweets(tweets_data)
columns_all_text = ['text', 'extended_tweet-full_text', 'retweeted_status-text', 
           'retweeted_status-extended_tweet-full_text', 'quoted_status-text', 
           'quoted_status-extended_tweet-full_text', 'lang', 'user-location', 
           'place-country_code']

# select text
tweets = select_text(tweets)
columns = ['text', 'lang', 'user-location', 'place-country', 
           'place-country_code', 'location-coordinates', 'user-screen_name']

# Create a DataFrame from `tweets`
df_tweets = pd.DataFrame(tweets, columns=columns)
# replaces NaNs by Nones
# df_tweets.where(pd.notnull(df_tweets), None, inplace=True)
#
df_tweets.head()

# 실행 코드

In [ ]:
import pandas as pd
import numpy as np
from glob2 import glob
import json

import nltk
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords as stp
from nltk.stem.porter import PorterStemmer

import gensim
import gensim.models.word2vec as w2v

from datetime import datetime
from datetime import date
from datetime import timedelta

In [ ]:
jsonFiles = glob('../pheme-rnr-dataset/**/source-tweet/*.json') #Can be used absolute or relative paths
print(jsonFiles)

In [ ]:
data_list=[]
for jsonFile in jsonFiles[0:300]:
    with open (jsonFile, 'r') as f:
        for l in f.readlines():
            if not l.strip (): # skip empty lines
                continue
            json_data = json.loads(l)
            # print (json_data,"\n\n")
            data_list.append(json_data)
tweet_data_frame = pd.DataFrame.from_dict(data_list)

In [ ]:
tweet_data_frame.info()

In [ ]:
def flatten_tweets(tweets):

    def capitalratio(tweet_text):
        uppers = [l for l in tweet_text if l.isupper()]
        capitalratio = len(uppers) / len(tweet_text)
        return capitalratio 

    def tweets2tokens(tweet_text):

        # Tokenizing
        tokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','', tweet_text.lower()))

        # Setting url value (whether the tweet contains http link) and filter http links
        url=0
        for token in tokens:
            if token.startswith( 'http' ):
                url=1
        tokens = [token for token in tokens if not token.startswith('http')]

        ## Stemming
        # porter = PorterStemmer()
        # tokens = [porter.stem(token) for token in tokens]

        # Filtering Stop words
        # from nltk.corpus import stopwords
        # stop_words = set(stopwords.words('english'))
        # tokens = [token for token in tokens if not token in stop_words]

        return tokens,url

    def contentlength(words):
        wordcount = len(words)
        return wordcount

    """ Flattens out tweet dictionaries so relevant JSON is in a top-level dictionary. """
    tweets_list = []
    total_tokens_l = []

    # Iterate through each tweet
    for tweet_obj in tweets:
        output_f = dict()

        output_f['text']= tweet_obj['text']
        
        output_f['text_token'], _ = tweets2tokens(tweet_obj['text'])
        total_tokens_l.extend(tweet_obj)


        output_f['char_count'] = len(output_f['text'])
        output_f['word_count'] = len(output_f['text_token'])

        output_f['has_question'] = "?" in output_f["text"]
        output_f['has_exclaim'] = "!" in output_f["text"]
        output_f['has_period'] = "." in output_f["text"]
        # hasperiod=sum(c =='.' for c in tweet_text)
            
    
        ''' User info'''
        # Store the user screen name in 'user-screen_name'
        # output_f['user-screen_name'] = tweet_obj['user']['screen_name']
        
        # Store the user location
        # output_f['user-location'] = tweet_obj['user']['location']

        acc_created = datetime.strptime(tweet_obj['user']['created_at'], '%a %b %d %H:%M:%S %z %Y')
        tweet_created = datetime.strptime(tweet_obj['created_at'], '%a %b %d %H:%M:%S %z %Y')
        age = (tweet_created - acc_created)
        # print(type(timedelta.total_seconds(age)))

        output_f['capital_ratio']=(capitalratio(tweet_obj['text']))

        # features=(capitalratio(data_list[0]['user']))
        output_f['tweet_count'] = np.log10(tweet_obj['user']['statuses_count'])
        output_f['listed_count'] = np.log10(tweet_obj['user']['listed_count'])
        output_f['follow_ratio'] = np.log10(tweet_obj['user']['followers_count'])
        output_f['age'] = int(timedelta.total_seconds(age)/86400)
        output_f['verified'] = tweet_obj['user']['verified']

        tweets_list.append(output_f)

    unk_tokens_l = list(set(total_tokens_l))
    print(len(total_tokens_l), len(unk_tokens_l)) # number of tokens and unique tokens

    return tweets_list

In [ ]:
df = pd.DataFrame(flatten_tweets(data_list))
df.head()
# for index, row in df.iterrows():
#     print(df.text[index],"\n")
# df.loc[df['has_period']==False]

In [ ]:
df.shape

In [ ]:
class MyCorpus():
    def __init__(self, train_data):
        self.train_data = train_data
        
    def __iter__(self):
        p = PorterStemmer()
        for i in range(len(self.train_data)):
            doc = self.train_data['text'][i]
            doc = re.sub(r'\S*@\S*\s?', '', doc, flags=re.MULTILINE) # remove email
            doc = re.sub(r'http\S+', '', doc, flags=re.MULTILINE) # remove web addresses
            doc = re.sub("\'", "", doc) # remove single quotes
            doc = remove_stopwords(doc)
            doc = p.stem_sentence(doc)
            words = simple_preprocess(doc, deacc=True)
            yield TaggedDocument(words=words, tags=[self.train_data['documentId'][i]])

---
### Experiment

In [437]:
for tweet_obj in data_list[0:3]:
    test_tx = tweet_obj['text']
    tokens = nltk.word_tokenize(re.sub(r'[([^\s\w]|_ |)','', test_tx))

In [439]:
tokens

['Soldier',
 'killed',
 'in',
 'Ottawa',
 'identified',
 'as',
 'Cpl',
 'Nathan',
 'Cirillo',
 'httptcoAOT1ZKyAei',
 'httptcon9NyPIQ5be']

In [ ]:
doc = re.sub(r'http\S+', '', doc, flags=re.MULTILINE) # remove web addresses


In [427]:
tokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','', test))

TypeError: expected string or bytes-like object

---

# Word2vec

In [ ]:
# brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['text'])
# brief_cleaning

In [ ]:
word2vec_  = w2v.Word2Vec(
    df['text_token'],
    sg = 1, 
    seed = 1,
    workers = 4,
    size = 300,
    min_count = 2,
    window = 5,
    sample = 1e-3 
)

In [ ]:
# word2vec_.build_vocab(df['text_token'])

In [ ]:
print("Number of word vectors: {}".format(len(word2vec_.wv.vocab)))

In [ ]:
df['text_token'][0][0] in word2vec_.wv.vocab

In [ ]:
word2vec_.wv.most_similar('extended')

In [ ]:
word2vec_.train(df['text_token'], total_examples = word2vec_.corpus_count, epochs = word2vec_.iter)

In [ ]:
word2vec_.wv.vectors.shape # vocab size / window size

In [ ]:
word2vec_.wv.most_similar('news')

In [ ]:
word_vectors = word2vec_.wv
vocabs = word_vectors.vocab.keys()

In [ ]:
print(len(vocabs))
print(len(list(word_vectors.vocab)))

In [ ]:
X = word_vectors[vocabs]
X

In [ ]:
w2v_vectors = word_vectors.vectors # here you load vectors for each word in your model
w2v_indices = {word: word_vectors.vocab[word].index for word in word_vectors.vocab} # here you load indices - with whom you can find an index of the particular word in your model 

In [ ]:
def vectorize(line): 
    words = []
    
    for word in line: # line - iterable, for example list of tokens 
        try:
            w2v_idx = w2v_indices[word]
        except KeyError: # if you does not have a vector for this word in your w2v model, continue 
            continue
        words.append(list(w2v_vectors[w2v_idx]))
        if not word:
            words.append(None)
        # print(words)

        if len(line) > len(words):
            continue
        # if word: 
            # words = np.asarray(words)
            # min_vec = words.min(axis=0)
            # max_vec = words.max(axis=0)
            # return np.concatenate((min_vec, max_vec))
    return np.asarray(words)
    # print(words)
    # print(len(line))
    # print(len(words))


## Word2Vec AVG

In [520]:
print("Tweet 1: ", df['text'][1])
print("Indice of '{}': {}".format(df['text_token'][1][0], w2v_indices[df['text_token'][1][0]]))
print("Indice of '{}': {}".format(df['text_token'][1][0], w2v_vectors[w2v_indices[df['text_token'][1][0]]]))
print("Indice of '{}': {}".format(df['text_token'][1][1], w2v_indices[df['text_token'][1][1]]))
print("Indice of '{}': {}".format(df['text_token'][1][1], w2v_vectors[w2v_indices[df['text_token'][1][1]]]))
print("\nVector of the first headline:\n", vectorize(df['text_token'][1]))

Tweet 1:  Police have clarified that there were two shootings in Ottawa today, not three: at the War Memorial and Parliament Hill.
Indice of 'police': 16
Indice of 'police': [ 0.10139903 -0.03351727  0.02891129 -0.04591792 -0.02251872  0.03903986
  0.00294215 -0.05639473  0.00608043  0.09522815  0.03273718  0.09814604
  0.11461891  0.11613031 -0.03141282 -0.01166822 -0.0044847   0.00797959
  0.00838375 -0.059598   -0.01984595 -0.01047864 -0.0394815  -0.01080784
  0.12610394  0.04355714 -0.02825906  0.00034501 -0.05115823  0.04958455
 -0.09103469 -0.00092993  0.01341081 -0.22750261  0.00929687  0.0981482
 -0.01679917 -0.01934557 -0.08163718 -0.03059717  0.08441687 -0.1119382
  0.1184299  -0.11451378 -0.03607124  0.06029971  0.00718947  0.0728694
  0.13608205 -0.1507328   0.10506455  0.07593489 -0.09304969 -0.17574425
 -0.05550925 -0.06192505  0.08443025  0.0756253   0.05127217 -0.09241643
  0.08867007  0.01600404 -0.0835479  -0.0095776  -0.06998151  0.13640194
  0.00058157  0.17830689 -

In [521]:
import copy
df['text_token_vec'] = copy.deepcopy(df['text_token'])

for index, sentence in enumerate(df['text_token_vec']):
    df['text_token_vec'][index] = vectorize(sentence).mean(axis=0)
    # df['text_token_vec'][0].mean(axis=0)

print(len(df['text_token_vec'][0]))
df[['text_token','text_token_vec']]

300


,text_token,text_token_vec
0,"[extended, dramatic, video, of, gunfire, insid...","[0.09440833, -0.031573307, 0.024093332, -0.039..."
1,"[police, have, clarified, that, there, were, t...","[0.1016643, -0.03303066, 0.02730084, -0.044099..."
2,"[soldier, killed, in, ottawa, identified, as, ...","[0.098197006, -0.032494348, 0.026898533, -0.04..."
3,"[norad, increases, number, of, planes, on, hig...","[0.0980182, -0.03213273, 0.025338322, -0.04129..."
4,"[all, 3, patients, injured, in, ottawashooting...","[0.10602436, -0.034144532, 0.027781254, -0.044..."
...,...,...
295,"[2, new, victims, en, route, to, ottawa, hospi...","[0.10297119, -0.033143934, 0.02643264, -0.0432..."
296,"[developing, story, here, gunman, killed, in, ...","[0.09752314, -0.031733863, 0.025908297, -0.042..."
297,"[police, confirm, 3rd, shooting, in, area, of,...","[0.100909874, -0.03269396, 0.02686235, -0.0435..."
298,"[gun, fire, exchange, in, parliament, hill, bu...","[0.099031426, -0.03231357, 0.026396746, -0.042..."


In [522]:
df_avg = pd.DataFrame(df['text_token_vec'].values.tolist()).add_prefix('token_avg')#.join(df)
df_avg

,token_avg0,token_avg1,token_avg2,token_avg3,token_avg4,token_avg5,token_avg6,token_avg7,token_avg8,token_avg9,...,token_avg290,token_avg291,token_avg292,token_avg293,token_avg294,token_avg295,token_avg296,token_avg297,token_avg298,token_avg299
0,0.094408,-0.031573,0.024093,-0.039885,-0.020761,0.034463,0.002402,-0.053406,0.005045,0.084720,...,-0.078363,-0.067360,-0.011084,-0.004991,0.086023,0.015997,-0.042468,0.009034,0.114680,-0.027427
1,0.101664,-0.033031,0.027301,-0.044099,-0.022456,0.037482,0.002724,-0.057728,0.006056,0.092435,...,-0.084164,-0.072562,-0.011624,-0.005453,0.094605,0.017698,-0.046654,0.010436,0.123407,-0.028992
2,0.098197,-0.032494,0.026899,-0.042544,-0.023967,0.037725,0.001601,-0.057752,0.005589,0.088875,...,-0.082158,-0.069797,-0.011306,-0.003078,0.090506,0.017486,-0.047874,0.012308,0.119222,-0.027772
3,0.098018,-0.032133,0.025338,-0.041297,-0.022105,0.034953,0.001899,-0.055017,0.004900,0.088602,...,-0.080414,-0.069475,-0.011104,-0.004861,0.088160,0.016586,-0.044894,0.009660,0.117827,-0.028134
4,0.106024,-0.034145,0.027781,-0.044783,-0.023270,0.038521,0.003041,-0.059217,0.006548,0.096233,...,-0.086878,-0.075664,-0.011687,-0.004443,0.096451,0.017912,-0.048035,0.010790,0.127890,-0.030124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.102971,-0.033144,0.026433,-0.043297,-0.022824,0.036866,0.002729,-0.056809,0.005611,0.093265,...,-0.084266,-0.073245,-0.010711,-0.005136,0.093951,0.017129,-0.045773,0.010227,0.123617,-0.029140
296,0.097523,-0.031734,0.025908,-0.042002,-0.021429,0.036109,0.003090,-0.055747,0.005454,0.088506,...,-0.081122,-0.069181,-0.011394,-0.004345,0.090020,0.016362,-0.045270,0.010196,0.117876,-0.027840
297,0.100910,-0.032694,0.026862,-0.043547,-0.022546,0.036593,0.002126,-0.056812,0.004750,0.091745,...,-0.083424,-0.072458,-0.011005,-0.005760,0.092761,0.017546,-0.045969,0.010313,0.122891,-0.028898
298,0.099031,-0.032314,0.026397,-0.042046,-0.022004,0.036505,0.002222,-0.056095,0.004911,0.089784,...,-0.082073,-0.070634,-0.011375,-0.004912,0.090059,0.017514,-0.044699,0.010109,0.119792,-0.028735


In [523]:
def document_vector(model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in model.wv.vocab]
    return np.mean(model[doc], axis=0)

## Doc2vec

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [442]:
df['text_token']

0      [extended, dramatic, video, of, gunfire, insid...
1      [police, have, clarified, that, there, were, t...
2      [soldier, killed, in, ottawa, identified, as, ...
3      [norad, increases, number, of, planes, on, hig...
4      [all, 3, patients, injured, in, ottawashooting...
                             ...                        
295    [2, new, victims, en, route, to, ottawa, hospi...
296    [developing, story, here, gunman, killed, in, ...
297    [police, confirm, 3rd, shooting, in, area, of,...
298    [gun, fire, exchange, in, parliament, hill, bu...
299    [soldier, shot, at, national, war, memorial, i...
Name: text_token, Length: 300, dtype: object

In [474]:
#Doc2vec 실행
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['text_token'])]
model = Doc2Vec(documents, vector_size=300, min_alpha=0.025, window=10, min_count=1, workers=4)

In [444]:
for epoch in range(10):
    if epoch % 2 == 0:
        print('Now training epoch %s '%epoch)
    model.train(documents, total_examples=model.corpus_count, epochs=1)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

Now training epoch 0 
Now training epoch 2 
Now training epoch 4 
Now training epoch 6 
Now training epoch 8 


In [485]:
# 첫번째 도큐먼트
documents[1]
# documents[199]

TaggedDocument(words=['police', 'have', 'clarified', 'that', 'there', 'were', 'two', 'shootings', 'in', 'ottawa', 'today', 'not', 'three', 'at', 'the', 'war', 'memorial', 'and', 'parliament', 'hill'], tags=[1])

In [484]:
# document vector의 정보
len(model.docvecs[0])

300

In [478]:
doctags = model.docvecs.doctags.items()
doctags = sorted(doctags, key=lambda x:x[1].offset)
doctags

[]

In [576]:
for idx, doctag in sorted(model.docvecs.doctags.items(), key=lambda x:x[1].offset):
    print(idx, doctag)

In [602]:
def most_similar_text(index):
    similar = model.docvecs.most_similar(index)

    print("The quried text: \n\n{} \n\nis most similar to the text:\n\n{}".format(df['text'][index],df['text'][similar[0][0]]))

similar = model.docvecs.most_similar(1)

# # for i, y in enumerate(similar):
#     # print("{}'s '"y[1])
# print("The quried text is most similar to the text\n\n{}".format(df['text'][similar[0][0]]))

In [604]:
most_similar_text(30)

The quried text: 

Senior U.S. official: Canadian government has informed U.S. that one shooter is dead in Ottawa. Live blog: http://t.co/q98AMohu7T 

is most similar to the text:

A Canadian soldier who was standing on guard for thee is gone. Our thoughts are w everyone affected by the shootings in Ottawa #CanadaStrong


In [590]:
similar[0][0]

280

In [577]:
vector = model.infer_vector(["system", "response"])

In [461]:
doc_vectors = model.docvecs
vocabs = doc_vectors.vocab.keys()

In [483]:
model.docvecs.wv

AttributeError: 'Doc2VecKeyedVectors' object has no attribute 'wv'

---

In [564]:
def document_vector(model, doc):
    # remove out-of-vocabulary words
    doc = [vec for vec in doc if vec in model.docvecs]
    return model[doc]

In [ ]:
document_vector(model, )

In [558]:
result = [(i, model.docvecs[i]) for i, j in enumerate(df['text_token'])]

In [573]:
list_doc = []
for index, tokens in enumerate(df['text_token']):
    list_doc.append(model.docvecs[index])

In [574]:
pd_doc = pd.DataFrame(list_doc).add_prefix('doc_vec')

In [575]:
pd_doc

,doc_vec0,doc_vec1,doc_vec2,doc_vec3,doc_vec4,doc_vec5,doc_vec6,doc_vec7,doc_vec8,doc_vec9,...,doc_vec290,doc_vec291,doc_vec292,doc_vec293,doc_vec294,doc_vec295,doc_vec296,doc_vec297,doc_vec298,doc_vec299
0,0.001174,-0.002209,0.001276,-0.001821,-0.000796,-0.000253,0.002193,-0.003003,0.000298,0.003488,...,-0.001035,-0.000250,0.001629,0.000343,0.002200,0.000286,-0.001029,0.001021,0.004979,-0.003107
1,0.000072,-0.000643,0.001412,-0.000262,-0.001581,-0.000478,0.002305,-0.000424,0.000885,0.002826,...,-0.002484,-0.001614,-0.000466,-0.001706,0.001469,-0.000877,-0.000901,0.000994,0.005010,-0.000370
2,-0.000422,0.000378,0.000531,0.000098,0.000671,0.001367,0.000605,0.000540,0.000643,-0.001153,...,-0.001668,-0.001710,0.001587,-0.001262,0.000716,-0.000444,0.000353,-0.001152,0.000395,0.000124
3,0.003968,-0.001037,-0.000448,-0.000493,-0.000122,0.001622,0.002108,-0.002218,0.000243,0.003003,...,-0.001377,-0.001745,0.001073,-0.001699,0.005931,-0.000040,-0.001468,-0.000238,0.008251,-0.003180
4,0.001743,-0.002971,0.002050,0.000865,-0.001889,-0.000670,0.001920,-0.003183,-0.000968,0.001418,...,-0.001513,-0.001383,0.000214,0.000514,0.005464,-0.000371,-0.001379,-0.000717,0.007901,-0.001406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.008864,-0.004854,0.002666,0.000149,-0.003610,0.003660,0.009176,-0.013051,-0.004851,0.012053,...,-0.008705,-0.006912,0.001525,-0.001280,0.019738,-0.004820,-0.009305,-0.003422,0.026781,-0.008457
296,0.005015,-0.002488,0.000023,-0.000325,-0.000288,0.001626,0.002102,-0.003017,-0.001379,0.002432,...,-0.005066,-0.001973,-0.000018,0.000307,0.007681,-0.001885,-0.004186,-0.000287,0.010256,-0.002379
297,0.008710,-0.005446,-0.000367,-0.000711,-0.003312,0.002145,0.007968,-0.008544,-0.003830,0.007356,...,-0.007832,-0.006358,-0.000395,0.000080,0.012673,-0.002346,-0.004022,-0.004028,0.020328,-0.006236
298,0.007198,-0.002597,0.000567,-0.000610,-0.001103,0.002521,0.004766,-0.005498,-0.001745,0.004327,...,-0.003289,-0.005891,-0.000416,-0.001990,0.008526,-0.000690,-0.005000,-0.000398,0.014628,-0.004342


In [561]:
d=pd.DataFrame(result[:][1])

In [526]:
df['doc_vec'] = np.asarray(result)

ValueError: Wrong number of items passed 2, placement implies 1

In [519]:
df['text_token_vec']

0      [1, [7.1845294e-05, -0.00064286514, 0.00141188...
1      [0.1016643, -0.03303066, 0.02730084, -0.044099...
2      [0.098197006, -0.032494348, 0.026898533, -0.04...
3      [0.0980182, -0.03213273, 0.025338322, -0.04129...
4      [0.10602436, -0.034144532, 0.027781254, -0.044...
                             ...                        
295    [0.10297119, -0.033143934, 0.02643264, -0.0432...
296    [0.09752314, -0.031733863, 0.025908297, -0.042...
297    [0.100909874, -0.03269396, 0.02686235, -0.0435...
298    [0.099031426, -0.03231357, 0.026396746, -0.042...
299    [0.09642967, -0.031837694, 0.027852055, -0.042...
Name: text_token_vec, Length: 300, dtype: object

In [492]:
doc = [word for word in doc if word in model.docvecs]

ValueError: DataFrame constructor not properly called!

In [ ]:
for index, sentence in enumerate(df['text_token_vec']):
    df['text_token_vec'][index] = vectorize(sentence).mean(axis=0)
    # df['text_token_vec'][0].mean(axis=0)

In [491]:
for index, sentence in enumerate(df['text']):
    df['doc_vec'][index] = 0

KeyError: 'doc_vec'

# Content based Features

In [ ]:
def capitalratio(tweet_text):
    uppers = [l for l in tweet_text if l.isupper()]
    capitalratio = len(uppers) / len(tweet_text)
    print("len of the text is: ",len(tweet_text))
    print("len of the upper is: ",len(uppers))
    return capitalratio

In [ ]:
features=dict()
twtx = data_list[1]['text']
features = capitalratio(twtx)

In [ ]:
features

## Word tokens

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
from nltk.corpus import stopwords as stp
# from textblob import TextBlob

In [ ]:
def tweets2tokens(tweet_text):
        tokens = nltk.word_tokenize(re.sub(r'([^\s\w]|_)+','', tweet_text.lower()))
        url=0
        for token in tokens:
            if token.startswith( 'http' ):
                url=1

        return tokens,url

In [ ]:
tokens,url=tweets2tokens(twtx)

In [ ]:
url

Transfer Leraning을 해야하나? 300개 디멘션이라는데